<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [2]:
# !pip3 install beautifulsoup4
# !pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
# assign the response to a object
resp = requests.get(static_url)


Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(resp.content, "html.parser")


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
soup.title

# Use soup.title attribute

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all( 'table')

Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []

tb_dt = first_launch_table.find_all('th')

for dt in tb_dt:

    name = extract_column_from_header(dt)
    
    if name:
        column_names.append(name)

column_names


['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

Check the extracted column names


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [12]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


#### Checking to see if every row has all 9 columns 

In [13]:
table_data = []
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    table_cols = []
    rows = table.find_all("tr")
    header_cols  = rows[0].find_all('th')
    # print(header_cols)
    
    for row in rows[1:]:  # Skip the header row
        if row.th and row.th.string:
            flight_number = row.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False
        
        if flag:
            row_data = {'Flight No.': flight_number}
            cells = row.find_all("td")
            row_data['cols'] = []
            for cell in cells:
                
            # row_data['cols'] = [cells[i]]
                cell_data = cell.get_text(strip=True)
                if cell_data:
                    
                    row_data['cols'].append(cell_data)
                else: 
                    row_data['cols'].append('null')

                    
            table_data.append(row_data)


In [14]:
for data in table_data:
    if len(data['cols']) !=9:
        print(data['cols'])
        
    print(data)
    
    data['cols'][2] = data['cols'][2].split(',')
    



{'Flight No.': '1', 'cols': ['4 June 2010,18:45', 'F9 v1.0[7]B0003.1[8]', 'CCAFS,SLC-40', 'Dragon Spacecraft Qualification Unit', 'null', 'LEO', 'SpaceX', 'Success', 'Failure[9][10](parachute)']}
{'Flight No.': '2', 'cols': ['8 December 2010,15:43[13]', 'F9 v1.0[7]B0004.1[8]', 'CCAFS,SLC-40', 'Dragondemo flight C1(Dragon C101)', 'null', 'LEO(ISS)', 'NASA(COTS)NRO', 'Success[9]', 'Failure[9][14](parachute)']}
{'Flight No.': '3', 'cols': ['22 May 2012,07:44[17]', 'F9 v1.0[7]B0005.1[8]', 'CCAFS,SLC-40', 'Dragondemo flight C2+[18](Dragon C102)', '525\xa0kg (1,157\xa0lb)[19]', 'LEO(ISS)', 'NASA(COTS)', 'Success[20]', 'No attempt']}
{'Flight No.': '4', 'cols': ['8 October 2012,00:35[21]', 'F9 v1.0[7]B0006.1[8]', 'CCAFS,SLC-40', 'SpaceX CRS-1[22](Dragon C103)', '4,700\xa0kg (10,400\xa0lb)', 'LEO(ISS)', 'NASA(CRS)', 'Success', 'No attempt']}
{'Flight No.': '5', 'cols': ['1 March 2013,15:10', 'F9 v1.0[7]B0007.1[8]', 'CCAFS,SLC-40', 'SpaceX CRS-2[22](Dragon C104)', '4,877\xa0kg (10,752\xa0lb)', 

In [20]:
extracted_row = 0

for row in table_data:
    launch_dict['Flight No.'].append(row['Flight No.'])
# for i, col in enumerate(row['cols']):

    cols = row['cols']
    
    # cols = row['cols'].split(', ')
    # print(dt)
    dt = str(cols[0]).split(',')
    # Handle cases where there's only a date without a time
    if len(dt) > 1:
        launch_dict['Date'].append(dt[0])
        launch_dict['Time'].append(dt[1].strip())
        
    else:
        launch_dict['Date'].append(dt[0])
        launch_dict['Time'].append('') 
        
        print(dt)
        
    # launch_dict['Date'].append(dt[0])
    # launch_dict['Time'].append(dt[1])
    
    bv = re.sub(r'\[\d+\]', '', str(cols[1]))
    print(bv)
    launch_dict['Version Booster'].append(bv)
    
    ls = re.sub(r'\[\d+\]', '', ','.join(cols[2]))
    print(ls)
    
    # launch_dict['Launch site'].append(ls)
    
    Payload = re.sub(r'\[\d+\]', '', cols[3])
    print(Payload)
    launch_dict['Payload'].append(Payload)
    
    Payloadmass = re.sub(r'\[\d+\]', '', cols[4])
    Payloadmass = re.sub(r'\xa0', '', Payloadmass)
    print(Payloadmass)
    
    launch_dict['Payload mass'].append(Payloadmass)
    
    orbit = re.sub(r'\[\d+\]', '', cols[5])
    print(orbit)
    launch_dict['Orbit'].append(orbit)
    
    cust = re.sub(r'\[\d+\]', '', cols[6])
    print(cust)
    launch_dict['Customer'].append(cust)
    
    outcome = re.sub(r'\[\d+\]', '', cols[7])
    print(outcome)
    launch_dict['Launch outcome'].append(outcome)
    
    bslanding = re.sub(r'\[\d+\]', '', cols[8])
    print(bslanding)
    launch_dict['Booster landing'].append(bslanding)
        
    extracted_row +=1 
    
        
        
        


F9 v1.0B0003.1
CCAFS,SLC-40
Dragon Spacecraft Qualification Unit
null
LEO
SpaceX
Success
Failure(parachute)
F9 v1.0B0004.1
CCAFS,SLC-40
Dragondemo flight C1(Dragon C101)
null
LEO(ISS)
NASA(COTS)NRO
Success
Failure(parachute)
F9 v1.0B0005.1
CCAFS,SLC-40
Dragondemo flight C2+(Dragon C102)
525kg (1,157lb)
LEO(ISS)
NASA(COTS)
Success
No attempt
F9 v1.0B0006.1
CCAFS,SLC-40
SpaceX CRS-1(Dragon C103)
4,700kg (10,400lb)
LEO(ISS)
NASA(CRS)
Success
No attempt
F9 v1.0B0007.1
CCAFS,SLC-40
SpaceX CRS-2(Dragon C104)
4,877kg (10,752lb)
LEO(ISS)
NASA(CRS)
Success
No attempt
F9 v1.1B1003
VAFB,SLC-4E
CASSIOPE
500kg (1,100lb)
Polar orbitLEO
MDA
Success
Uncontrolled(ocean)[d]
F9 v1.1B1004
CCAFS,SLC-40
SES-8
3,170kg (6,990lb)
GTO
SES
Success
No attempt
F9 v1.1
CCAFS,SLC-40
Thaicom 6
3,325kg (7,330lb)
GTO
Thaicom
Success
No attempt
F9 v1.1
Cape Canaveral,LC-40
SpaceX CRS-3(Dragon C105)
2,296kg (5,062lb)
LEO(ISS)
NASA(CRS)
Success
Controlled(ocean)[d]
F9 v1.1
Cape Canaveral,LC-40
Orbcomm-OG2-1(6 satellites)


In [21]:
print(f'{extracted_row}')
launch_dict

121


{'Flight No.': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121',
  '1'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [22]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [23]:
df.shape

(242, 11)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [24]:
df.to_csv('spacex_web_scraped.csv', index=False)


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
